### 1. Web Scraping

I implemented a web scraper that scans through all 556 pages of https://www.tcgcollector.com/cards/intl, extracts all 13888 individual card urls like https://www.tcgcollector.com/cards/16633/weedle-vivid-voltage-001-185, scrapes static and dynamic content from the individual card pages, parses the information into a structured format, and stores structured entries in csv files. The web scraper is built with Scrapy, a python based web scraping framework, and Selenium, which is suitable for scraping dynamic web content. 

The scraped dataset has 13888 rows and 18 columns ('attacks', 'card_format', 'card_number', 'card_type', 'damages','energy', 'evolution', 'evolved_from', 'expansion', 'hp','illustrators', 'languages', 'name', 'rarity', 'resistance','retreat_cost', 'variants', 'weakness'). Gnerally speaking, fields with only one value are stored in dataset as is, while fields with more than one values are combined into one semicolon separated string to be stored. Not all available fields from the website are scraped for now due to time constriant (for example, card effect, card rule, additional information, provides, note the last one is for energy cards), but we can easily add these fields to the scraper. 

Among the current columns, 'variants' and 'languages' are scraped from dynamic content while the other fields are scraped statically. Static scraping is mostly done by parsing response object using xpath, while dynamic scraping is achieved with simulated page clicks followed by parsing. To access dynamic content, we also need to provide login credentials of a premium account. 

Scrapy is a powerful framework that handles web scraping in a distributed way, so row indices of the scraped dataset don't reflect the order of corresponding cards in the original website. In order to perserve ordering of cards in the website, we need to do some further processing. 

### 2. Load Data

We will now load scraped data in a pandas dataframe, print out its shape and display the first few lines. 

In [1]:
import pandas as pd

df1 = pd.read_csv('data/1-200.csv', encoding='utf-8')
df2 = pd.read_csv('data/201-400.csv', encoding='utf-8')
df3 = pd.read_csv('data/401-556.csv', encoding='utf-8')
df = pd.concat([df1, df2, df3], ignore_index=True)

In [2]:
df.shape

(13888, 18)

In [3]:
df.head()

,attacks,card_format,card_number,card_type,damages,energy,evolution,evolved_from,expansion,hp,illustrators,languages,name,rarity,resistance,retreat_cost,variants,weakness
0,Magical Shot;Swelling Pulse,Standard,016/073,Pokémon V,30;120+,Psychic,Basic,NaN,Champion's Path,210.0,Kagemaru Himeno,English (EN);French (FR);Spanish (ES);Portugue...,Gardevoir V,Rare Holo V,NaN,2.0,Unlimited,Metal×2
1,Electrified Bite Mark;Electro Sprinkler,Standard,059/185,Pokémon,60;120,Lightning,Stage 2,Eelektrik,Vivid Voltage,160.0,tetsuya koizumi,English (EN);French (FR);Spanish (ES);Portugue...,Eelektross,Rare,NaN,3.0,Unlimited;Reverse Holo,Fighting×2
2,NaN,Standard,201/185,"Full Art Trainer,Item",NaN,NaN,NaN,NaN,Vivid Voltage,NaN,Toyste Beach,English (EN);French (FR);Spanish (ES);Portugue...,Hero's Medal,Rare Secret,NaN,NaN,Unlimited,NaN
3,Fire Mane,Standard,026/185,Pokémon,100,Fire,Stage 1,Eevee,Vivid Voltage,110.0,Eri Yamaki,English (EN);French (FR);Spanish (ES);Portugue...,Flareon,Rare,NaN,2.0,Unlimited;Reverse Holo,Water×2
4,Auspicious Tone;Hypnoblast,Standard,072/185,Pokémon,0;30,Psychic,Basic,NaN,Vivid Voltage,70.0,Asako Ito,English (EN);French (FR);Spanish (ES);Portugue...,Chimecho,Common,Fighting-30,1.0,Unlimited;Reverse Holo,Darkness×2


### 3. Explode

We will fill missing values in variants and languages with empty strings, transform each semicolon separated string in 'variants' and 'languages' into a list of strings, and use the built-in explode method to generate a unique row for each variant (printing) and language card. 

In [4]:
# explode variants
df["variants"] = df["variants"].fillna("")
df["variants"] = df["variants"].apply(lambda x: x.split(";"))
df = df.explode("variants", ignore_index=True)

The dataframe after exploding for variants contains 25517 rows, and we will display the first few lines. 

In [5]:
df.shape

(25517, 18)

In [6]:
df.head()

,attacks,card_format,card_number,card_type,damages,energy,evolution,evolved_from,expansion,hp,illustrators,languages,name,rarity,resistance,retreat_cost,variants,weakness
0,Magical Shot;Swelling Pulse,Standard,016/073,Pokémon V,30;120+,Psychic,Basic,NaN,Champion's Path,210.0,Kagemaru Himeno,English (EN);French (FR);Spanish (ES);Portugue...,Gardevoir V,Rare Holo V,NaN,2.0,Unlimited,Metal×2
1,Electrified Bite Mark;Electro Sprinkler,Standard,059/185,Pokémon,60;120,Lightning,Stage 2,Eelektrik,Vivid Voltage,160.0,tetsuya koizumi,English (EN);French (FR);Spanish (ES);Portugue...,Eelektross,Rare,NaN,3.0,Unlimited,Fighting×2
2,Electrified Bite Mark;Electro Sprinkler,Standard,059/185,Pokémon,60;120,Lightning,Stage 2,Eelektrik,Vivid Voltage,160.0,tetsuya koizumi,English (EN);French (FR);Spanish (ES);Portugue...,Eelektross,Rare,NaN,3.0,Reverse Holo,Fighting×2
3,NaN,Standard,201/185,"Full Art Trainer,Item",NaN,NaN,NaN,NaN,Vivid Voltage,NaN,Toyste Beach,English (EN);French (FR);Spanish (ES);Portugue...,Hero's Medal,Rare Secret,NaN,NaN,Unlimited,NaN
4,Fire Mane,Standard,026/185,Pokémon,100,Fire,Stage 1,Eevee,Vivid Voltage,110.0,Eri Yamaki,English (EN);French (FR);Spanish (ES);Portugue...,Flareon,Rare,NaN,2.0,Unlimited,Water×2


In [7]:
# explode languages
df["languages"] = df["languages"].fillna("")
df["languages"] = df["languages"].apply(lambda x: x.split(";"))
df = df.explode("languages", ignore_index=True)

The dataframe after exploding for languages contains 278657 rows, and we will display the first few lines.

In [8]:
df.shape

(278657, 18)

In [9]:
df.head()

,attacks,card_format,card_number,card_type,damages,energy,evolution,evolved_from,expansion,hp,illustrators,languages,name,rarity,resistance,retreat_cost,variants,weakness
0,Magical Shot;Swelling Pulse,Standard,016/073,Pokémon V,30;120+,Psychic,Basic,NaN,Champion's Path,210.0,Kagemaru Himeno,English (EN),Gardevoir V,Rare Holo V,NaN,2.0,Unlimited,Metal×2
1,Magical Shot;Swelling Pulse,Standard,016/073,Pokémon V,30;120+,Psychic,Basic,NaN,Champion's Path,210.0,Kagemaru Himeno,French (FR),Gardevoir V,Rare Holo V,NaN,2.0,Unlimited,Metal×2
2,Magical Shot;Swelling Pulse,Standard,016/073,Pokémon V,30;120+,Psychic,Basic,NaN,Champion's Path,210.0,Kagemaru Himeno,Spanish (ES),Gardevoir V,Rare Holo V,NaN,2.0,Unlimited,Metal×2
3,Magical Shot;Swelling Pulse,Standard,016/073,Pokémon V,30;120+,Psychic,Basic,NaN,Champion's Path,210.0,Kagemaru Himeno,Portuguese (PT),Gardevoir V,Rare Holo V,NaN,2.0,Unlimited,Metal×2
4,Magical Shot;Swelling Pulse,Standard,016/073,Pokémon V,30;120+,Psychic,Basic,NaN,Champion's Path,210.0,Kagemaru Himeno,Italian (IT),Gardevoir V,Rare Holo V,NaN,2.0,Unlimited,Metal×2


### 4. Groupby

Before doing the groupby, let's first create a new column for max attack damage of each card. 

In [10]:
import re
def process_damage(damages):
    dams = damages.split(";")
    # keeps numeric characters only
    dams = [re.sub('[^0-9]', '', dam) for dam in dams]
    # computes largest damage
    largest = 0
    for dam in dams:
        # skips empty values
        if dam=="": continue
        largest = max(largest, int(dam))
    return largest

In [11]:
df["damages"] = df["damages"].fillna("")
df["max_damage"] = df["damages"].apply(process_damage)

Now we will group the dataframe by {name, expansion} (corresponding to pokemon name and set name), and sort in descending order by maximum attack damage. 

In [12]:
df_grouped = df.groupby(by=["name", "expansion"])["max_damage"].max()
df_grouped = df_grouped.sort_values(ascending=False)

Let's display the first few lines of the groupby object for a quick sanity check. 

In [13]:
df_grouped.head()

name                            expansion           
Zoroark and Legendary Pokémon   Unnumbered Promos       1000
Tornadus, Thundurus & Landorus  Unnumbered Promos       1000
Charizard-GX                    SM Black Star Promos     300
Charizard VMAX                  Champion's Path          300
                                Darkness Ablaze          300
Name: max_damage, dtype: int64

### 5. To Dataframe

We will now convert the groupby object into a pandas dataframe, and keep only "name" and "expansion" as columns.

In [14]:
df_grouped = df_grouped.reset_index().iloc[:, :2]

This is how the dataframe looks like:

In [15]:
df_grouped.head()

,name,expansion
0,Zoroark and Legendary Pokémon,Unnumbered Promos
1,"Tornadus, Thundurus & Landorus",Unnumbered Promos
2,Charizard-GX,SM Black Star Promos
3,Charizard VMAX,Champion's Path
4,Charizard VMAX,Darkness Ablaze


### 6. Cache Dataframe

Finally, we will write cached dataframe to a csv file. 

In [16]:
df_grouped.to_csv("data/output.csv", index=False)

End of everything! Thanks for reading. 